In [13]:
# Everything in one clode block

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import chardet

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

# Function to detect file encoding
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']


# Directories
input_dir = 'data2'                 # Data2 is folder which only contains bus 1
output_dir = 'results2'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over each file in the input directory
for file_name in os.listdir(input_dir):
    # Construct full file path
    file_path = os.path.join(input_dir, file_name)
    
    # Check if it is a file
    if os.path.isfile(file_path):
        try:
            # Detect the encoding of the file
            encoding = detect_encoding(file_path)
            
            # Load the data from the file with detected encoding
            df = pd.read_csv(file_path, encoding=encoding)

            # Extract bus number from file name (assuming file name format is consistent)
            bus_number = file_name.split('_')[1]
            
            # Create a subdirectory for each bus number
            bus_dir = os.path.join(output_dir, f'bus_{bus_number}')
            if not os.path.exists(bus_dir):
                os.makedirs(bus_dir)



            # 1. Read CSV file and set datetime index.
            start_date = '01-01-2018'
            datetime_index = pd.date_range(start=start_date, periods=len(df), freq='h')
            df.index = datetime_index


            # 2. Create time-based features.

            # Feature creation
            def create_features(df):
                """
                Create time series features based on time series index.
                """
                df = df.copy()
                df['hour'] = df.index.hour
                df['dayofweek'] = df.index.dayofweek
                df['quarter'] = df.index.quarter
                df['month'] = df.index.month
                df['dayofyear'] = df.index.dayofyear
                df['dayofmonth'] = df.index.day
                return df

            df = create_features(df)
            

            # 3. Lag features

            def add_lags(df):
                target_map = df['Load'].to_dict()

                df['lag1'] = (df.index - pd.Timedelta('1 hour')).map(target_map)
                df['lag24'] = (df.index - pd.Timedelta('24 hour')).map(target_map)
                df['lag100'] = (df.index - pd.Timedelta('3 days')).map(target_map)
                df['lag104'] = (df.index - pd.Timedelta('7 days')).map(target_map)
                df['lag105'] = (df.index - pd.Timedelta('14 days')).map(target_map)
                return df
            df = add_lags(df)
            

            # 4. Prepare the data for training and validation.

            # Review train/test split
            train = df.loc[(df.index < '09-01-2018') | ((df.index >= '9-24-2018') & (df.index < '10-01-2018'))]           # Train including last 7 days before test start
            validation = df.loc[(df.index >= '09-01-2018') & (df.index < '9-24-2018')]
            test = df.loc[(df.index >= '10-01-2018')] # & (df.index < '12-31-2018')]     # Index 6554

            

            # Create the model
            train = create_features(train)
            validation = create_features(validation)
            test = create_features(test)

            FEATURES = ['lag104', 'lag1', 'lag24', 'lag100', 'lag105']
            TARGET = 'Load'

            X_train = train[FEATURES]
            y_train = train[TARGET]

            X_val = validation[FEATURES]
            y_val = validation[TARGET]

            X_test = test[FEATURES]
            y_test = test[TARGET]

            # Save the DataFrame as a CSV file in the bus subdirectory
            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_X_train.csv')
            X_train.to_csv(csv_path, index=False)
            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_y_train.csv')
            y_train.to_csv(csv_path, index=False)

            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_X_val.csv')
            X_val.to_csv(csv_path, index=False)
            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_y_val.csv')
            y_val.to_csv(csv_path, index=False)

            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_X_test.csv')
            X_test.to_csv(csv_path, index=False)
            csv_path = os.path.join(bus_dir, f'bus_{bus_number}_y_test.csv')
            y_test.to_csv(csv_path, index=False)


            # 5. Train the XGBoost model.



            reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                                   n_estimators=1000,
                                   early_stopping_rounds=50,
                                   objective='reg:squarederror',
                                   max_depth=3,                         # Used to be 3
                                   learning_rate=0.1)                   # Used to be 0.01
            reg.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],      # Removed eval_metric='rmse'
                verbose=50                      # Frequency of printing validation
            )


            # 6. Plot the actual vs predicted values.

            # Feature importance
            fi = pd.DataFrame(data=reg.feature_importances_,        # feature_importances_ is a standard function of "reg"
                         index=reg.feature_names_in_,
                         columns=['importance'])
            
            # fi.sort_values('importance').plot(kind='barh', title='Feature Importance')  # Sort by importance
            # Save the plot as an image file in the bus subdirectory

            ax = os.path.join(bus_dir, 'feature_importance_sorted.png')
            plt.savefig(ax)
            plt.clf()  # Clear the current figure to prevent overlap in the next iteration

            # # Save the plot as an image file in the bus subdirectory
            # plot_path = os.path.join(bus_dir, 'feature_importance')
            # plt.savefig(plot_path)
            # plt.clf()  # Clear the current figure to prevent overlap in the next iteration
            
            fi_sorted = fi.sort_values('importance', ascending=False)
            csv_path = os.path.join(bus_dir, 'plotfeature_importance_sorted.csv')
            fi.to_csv(csv_path, index=False)


            # Plot everything
            import pandas as pd
            import os
            from sklearn.metrics import mean_squared_error
            import numpy as np
            # Forecast on test
            test['prediction'] = reg.predict(X_tes)
            results = test[['Load', 'prediction']]  # Check this
            csv_path = os.path.join(bus_dir, 'test_predictions.csv')
            subset.to_csv(csv_path, index=True)

            df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
            ax = df[['Load']].plot(figsize=(15, 5))
            df['prediction'].plot(ax=ax, style='.')
            plt.legend(['Truth Data', 'Predictions'])
            ax.set_title('Raw Data and Prediction')
            plot_path = os.path.join(bus_dir, 'plot_year.png')
            plt.savefig(plot_path)
            plt.clf()  # Clear the current figure to prevent overlap in the next iteration

            # Calculate Mean Squared Error all year
            score = np.sqrt(mean_squared_error(test['Load'], test['prediction']))**2
            score_df = pd.DataFrame({'MSE': [score]})           # Create a DataFrame to store the score
            csv_path = os.path.join(bus_dir, 'MSE_year.csv')
            score_df.to_csv(csv_path, index=False)



            # Plot for 24 hours

            # Define the date range for the plot
            start_date = '2018-10-01'
            end_date = '2018-10-02'

            # Filter the data for the specified date range
            subset = df.loc[(df.index > start_date) & (df.index < end_date)]

            # Plot the Load data
            ax = subset['Load'].plot(figsize=(15, 5), title='Predictions vs actual Load 1 okt 2018')

            # Plot the Prediction data
            subset['prediction'].plot(style='.', ax=ax)

            # Set the legend
            plt.legend(['Truth Data', 'Prediction'])

            # Set the x-ticks and x-tick labels
            xticks = subset.index[::1]  # For example, take every 6th index as a tick
            ax.set_xticks(xticks)
            ax.set_xticklabels(xticks.strftime('%H'), rotation=45, ha='right')     # ('%m-%d %H:%M')

            # Set the x and y axis labels
            ax.set_xlabel('Time in Hours')
            ax.set_ylabel('Load (MW)')

            # Save the plot as an image file in the bus subdirectory
            ax = os.path.join(bus_dir, 'plot_24_hours.png')
            plt.savefig(ax)
            plt.clf()  # Clear the current figure to prevent overlap in the next iteration
            

            # plot_path = os.path.join(bus_dir, 'feature_importance')
            # plt.savefig(plot_path)
            # plt.clf()  # Clear the current figure to prevent overlap in the next iteration



            # # Calculate Mean Squared Error 24 hours
            # score = np.sqrt(mean_squared_error(test['Load'], test['prediction']))**2
            # score_df = pd.DataFrame({'MSE': [score]})           # Create a DataFrame to store the score
            # bus_dir = '.'                                       # Update this to your actual directory path if different
            # csv_path = os.path.join(bus_dir, 'MSE_24_hours.csv')
            # score_df.to_csv(csv_path, index=False)



            # # Save the plot as an image file in the bus subdirectory
            # plot_path = os.path.join(bus_dir, 'plot.png')
            # plt.savefig(plot_path)
            # plt.clf()  # Clear the current figure to prevent overlap in the next iteration
            
            # # Save the DataFrame as a CSV file in the bus subdirectory
            # csv_path = os.path.join(bus_dir, f'bus_{bus_number}_data.csv')
            # df.to_csv(csv_path, index=False)
            
            # ax = df.loc[(df.index > '10-01-2018') & (df.index < '10-04-2018')]['Load'] \
            #     .plot(figsize=(15, 5), title='3 days of data')
            # df.loc[(df.index > '10-01-2018') & (df.index < '10-04-2018')]['prediction'] \
            #     .plot(style='.')
            # plt.legend(['Truth Data','Prediction'])

            # # Save the plot as an image file in the bus subdirectory
            # plot_path = os.path.join(bus_dir, 'plot_3_days.png')
            # plt.savefig(plot_path)
            # plt.clf()  # Clear the current figure to prevent overlap in the next iteration



        except Exception as e:
            print(f"Error processing file {file_name}: {e}")


[0]	validation_0-rmse:177.39285
[50]	validation_0-rmse:1.22115
[100]	validation_0-rmse:0.74382
[150]	validation_0-rmse:0.67076
[200]	validation_0-rmse:0.62989
[250]	validation_0-rmse:0.60207
[300]	validation_0-rmse:0.58509
[350]	validation_0-rmse:0.57449
[400]	validation_0-rmse:0.56476
[450]	validation_0-rmse:0.55079
[500]	validation_0-rmse:0.54652
[550]	validation_0-rmse:0.53922
[600]	validation_0-rmse:0.53111
[650]	validation_0-rmse:0.52652
[700]	validation_0-rmse:0.52220
[750]	validation_0-rmse:0.51790
[800]	validation_0-rmse:0.51623
[850]	validation_0-rmse:0.51129
[900]	validation_0-rmse:0.51039
[950]	validation_0-rmse:0.51179
[999]	validation_0-rmse:0.50765


<Figure size 640x480 with 0 Axes>

<Figure size 1500x500 with 0 Axes>